In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('seaborn-bright')

import pickle
from sklearn import linear_model
from sklearn import metrics

from IPython.display import display
%load_ext memory_profiler
%load_ext line_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
# not in notes (!)
df = pd.DataFrame(np.ones(50).reshape(10,5), 
                      index=pd.date_range('1/1/2016', periods=10, freq='H' ),
                      columns=list('ABCDE'))
df['size']=0
df['size'] = pd.Series([0,0,0,0,200,0,0,2000,0,0], index=df.index)
df.loc[:,"size_csum"] = df['size'].cumsum()

start = pd.to_datetime('1/1/1900')
df.loc[:,"size_astime"] = start + pd.to_timedelta(df.size_csum,'ms')
df.index = df.size_astime

display(df)
print(df.E.resample('1000ms',label='right').sum())

# timestamp_col = df.timestamp.resample('30min',label='right').last()

# pd.to_datetime(timestamp_col).dropna()

# after going around the circle, the workaround has to be applying an extra operation:
# `pd.to_datetime(timestamp_col).dropna()`

# df

In [ ]:
# flattening history for autoregression
df = pd.DataFrame({'val': [10,40,30,10,11,12,10,11,12,19]}, index=pd.date_range('2016-01-01 19:00:00', periods=10, freq='H' ) )
display(df)
def myf(df):
    for i in range(1,2):
        df['val_t-' + str(i)] = df['val'].shift(i)

    for i in range(1,5):
        df['d_val_t+' + str(i)] = df['val'].shift(-i) - df['val']

    df[['d_val_t+3','d_val_t+4']].mean(axis=1)
    return df

df.groupby(pd.TimeGrouper('D')).apply(myf)

In [23]:
%pdb off
def myf(lm, **args):
    args['cv'] = 'hep'
    print args
    model = lm(**args)
    X_train = [[1,2,2],[3,4,4]]
    y_train = [100,200]
    
    result = model.fit(X_train, y_train)  
    return result

my_model = Ridge
res = myf(my_model, alpha=0.1)
print res.coef_

my_model = Lasso
res = myf(my_model)
print res.coef_

Automatic pdb calling has been turned OFF
{'alpha': 0.1, 'cv': 'hep'}


TypeError: __init__() got an unexpected keyword argument 'cv'

In [86]:
X_train = [[20,10,1],[22,11,2],[33,13,3],[-20,-10,1],[-22,-11,2],[-33,-13,3]]
y_train = [-1,0,1,-1,0,1]
# y_train = ['-1','0','1','-1','0','1']

model = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg', class_weight={-1:0.5, 0:0, 1:0.5})

model.fit(X_train, y_train)

# model.predict([15,8,3])
# model.score([[15,8,3],[2,3,3],[6,6,6],[6,6,7]],[1,-1,0,0],[1,1,0,0])
model.score(X_train,y_train)


# model.coef_

0.66666666666666663

In [45]:
m = linear_model.LogisticRegression()

In [48]:
m.__class__.__name__

'LogisticRegression'

In [74]:
np.round([2.2,2.1])
q=np.array([2.2,2.1])
q.astype(int)

array([2, 2])

In [109]:


# my_scorer

X_train = [[20,10,1],[22,11,2],[33,13,3],[-20,-10,1],[-22,-11,2],[-33,-13,3]]
y_train = [-1,0,1,-1,0,1]
# y_train = ['-1','0','1','-1','0','1']

my_scorer = metrics.make_scorer(metrics.accuracy_score, sample_weight=[1,0,0])

model = linear_model.LogisticRegressionCV(multi_class='multinomial', solver='newton-cg', cv=2, scoring=my_scorer)

model.fit(X_train, y_train)

# model.predict([15,8,3])
# model.score([[15,8,3],[2,3,3],[6,6,6],[6,6,7]],[1,-1,0,0],[1,1,1,1])
# model.score(X_train,y_train)


# model.coef_


LogisticRegressionCV(Cs=10, class_weight=None, cv=2, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=None, refit=True,
           scoring=make_scorer(accuracy_score, sample_weight=[1, 0, 0]),
           solver='newton-cg', tol=0.0001, verbose=0)

In [138]:
# custom scorer for scoring cv
X_train = [[20,10,1],[22,11,2],[33,13,3],[-20,-10,1],[-22,-11,2],[-33,-13,3]] # pattern: z=1,2,3 is labeled -1,0,1 respectively
y_train = [-1,0,1,-1,0,1]

scorer = metrics.make_scorer(metrics.accuracy_score, sample_weight=[0.5])

model = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
model.fit(X_train, y_train)

def my_scorer(model,X, y_true):
    y_pred = model.predict(X)

    temp_weight = y_true
    ind = y_true == 0
    temp_weight[ind] = 0
    temp_weight[~ind] = 1 # weight want to be integers

    return metrics.accuracy_score(y_true, y_pred, sample_weight=temp_weight )

print scorer(model,[[1,2,3]],[1]) # same interface as this
my_scorer(model,[[1,2,3]],[1]) 

1.0


1.0